# Timing and Measuring Throughput

In [2]:
import pyopencl as cl
import pyopencl.array
import pyopencl.clrandom
import numpy as np
import numpy.linalg as la
from time import time
import matplotlib.pyplot as plt

In [4]:
ctx = cl.create_some_context(interactive=True)
queue = cl.CommandQueue(ctx)

In [5]:
n = 10**8 * 2
x = cl.clrandom.rand(queue, n, np.float32)

In [6]:
intensities = []
timings = []
ops = []

for nops in range(4, 48, 4):
    print(nops)

    lines = ["op_t xi = x[i];", "op_t v = xi;"]
    for n in range(nops):
        lines.append("v = xi*(xi+v);")
        
    operations = 2*nops
    ops.append(operations)
    intensity = operations / x.dtype.itemsize
    intensities.append(intensity)
    
    src = """
    typedef float op_t;
    __kernel void perftest(__global op_t *x, __global op_t *output)
    {
      int i = get_global_id(0);
      %s
      output[i] = v;
    }
    """ % ("\n".join(lines))
    #print(src)
    prg = cl.Program(ctx, src).build()
    knl = prg.perftest
    
    output = cl.array.empty_like(x)
    for i in range(3):
        knl(queue, x.shape, (512,), x.data, output.data)
        
    nruns = 10
    queue.finish()
    start_t = time()
    for i in range(nruns):
        knl(queue, x.shape, (512,), x.data, output.data)
    queue.finish()
    timings.append((time()-start_t)/nruns)

In [7]:
plt.figure(figsize=(10, 6))
plt.xlabel("Intensity")
plt.ylabel("GFlops/s")

timings = np.array(timings)
gflops = np.array(ops)*x.size/1e9/timings

plt.plot(intensities, gflops, "o-")